# 

In [4]:
import openai
from openai import OpenAI

# Point to the server
client = OpenAI(base_url="http://localhost:8000/v1", api_key="cltl")

In [5]:
# Sentences to classify
sentences = [
    "I hate you and I hope you fail.",
    "What a beautiful day to go for a walk!",
    "Your idea is stupid and nobody cares."
]

sentences = [
    "#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon @USER URL",
    "#ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.",
]

# Build a single prompt
prompt = "Classify each of the following sentences as 'hate' or 'non-hate':\n\n"
for i, s in enumerate(sentences, 1):
    prompt += f"{i}. {s}\n"

prompt += "\nReturn the results in the format:\n<number>. <label>\n"

# Make one request for all sentences
response = client.completions.create(
    model="local model",  # currently unused
    prompt=prompt,
    max_tokens=50,
    temperature=0,
    stop=["Classify", "\n\n"]
)

# Print the raw model output
print(response.choices[0].text.strip())

1. hate
2. non-hate


In [6]:
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
import pandas as pd

# evaluation data
# 15923,"#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon ⁦@USER URL",1

data = pd.read_csv("Subjectivity_mining_assignment_4_5_data/olid-test.csv")
data.head()

,id,text,labels
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,1
1,27014,"#ConstitutionDay is revered by Conservatives, ...",0
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,0
3,13876,#Watching #Boomer getting the news that she is...,0
4,60133,#NoPasaran: Unity demo to oppose the far-right...,1


In [214]:
#TODO: changed, since it only had the offensive nonoffensive parts
def evaluate(prompt_info, data):
    ground_truth = []

   
    prompt = prompt_info['text'] + f"Classify each of the following sentences as '{prompt_info['labels'][0]}' or '{prompt_info['labels'][1]}':\n\n"

    i = 1
    for id, row in data.iterrows():
        ground_truth.append(row['labels'])
        prompt += f"{i}. {row['text']}\n"
        i+=1
    
    prompt += "\nReturn the results in this format:\n<number>. <label>\n"

   
    print("----- Prompt Sent -----")
    print(prompt)

    #make one request for all sentences
    response = client.completions.create(
        model="local model",  
        prompt=prompt,
        max_tokens=200,
        temperature=0,
        stop=["Classify", "\n\n"]
    )


    raw_output = response.choices[0].text.strip()
    predicted = raw_output.split("\n")

    print("----- Raw Model Output -----")
    print(predicted)

    final_predictions = []

    for p in predicted:
        try:
            p = p.split(". ")[1].strip().lower()  
        except:
            print("⚠️ Skipped line:", p)
            continue

        #! TODO: this is a bit changed to create the lablles option
        if p == prompt_info['labels'][0]:
            final_predictions.append(1)
        elif p == prompt_info['labels'][1]:
            final_predictions.append(0)
        else:
            print("⚠️ Unexpected label:", p)
            continue

    return final_predictions, ground_truth


In [215]:
# get 10:20 from data
data[10:20]

,id,text,labels
10,22452,𝓒-𝓒𝓮𝓭𝓻𝓪𝓷.. The Nord cannot make a single move...,0
11,15565,#ConsTOO THE PLACE FOR FED UP CONSERVATIVES !!!,0
12,64376,#GreatAwakening #QAnon #PatriotsUnited #WWG1WG...,0
13,12588,@USER Nigga ware da hits at,1
14,34263,#StopKavanaugh he is liar like the rest of the...,1
15,65773,#Dayspromo week of September 17th #days #dool ...,0
16,95457,#BiggBossTelugu2 Let’s all include @USER in al...,0
17,24930,#Dallas#Policewoman kills neighbor in HIS own ...,0
18,15938,Always smack URL,0
19,45712,"#BREAKING: After a week-long trial, a Linn Cou...",0


In [216]:
# Prompts 
!#TODO: changed

prompts = {
    "prompt1": {
        "text": "", # already in function itself
        "labels": ['offensive', 'non-offensive']},
    "prompt2": {
        "text": """You are a classifier of hate speech. The following data contains language that is either non offensive or offensive. 
        Please note these are the definitions of offensive/non-offensive so try to stick to these guidelines.
        'offensive': A post that contains insults, threats, or profanity directed towards a person or group.
        'non-offensive': A post that is neutral, polite, or does not contain any harmful language.""",
        "labels": ['offensive', 'non-offensive']},
    "prompt3": {
        "text": """You are a classifier of hate speech. The following data contains language that is either non-offensive or offensive. 
        Please note these are the definitions of offensive/non-offensive, so try to stick to these guidelines. 
        'offensive': all types of expression that incite, promote, spread or justify violence, hatred or discrimination against a person or group of persons, or that denigrates them, by reason of their real or attributed personal characteristics or status such as "race", color, language, religion, nationality, national or ethnic origin, age, disability, sex, gender identity and sexual orientation. 
        'non-offensive': Negation of offensive definition.""",
        "labels": ['offensive', 'non-offensive']}
    ,
    "prompt4": {
        "text":"""You are a classifier of hate speech. The following data contains language that is either harmful or safe.
        Please note these are the definitions of harmful/safe, so try to stick to these guidelines.
        'harmful': Language that is offensive, abusive, or hateful, and that can cause emotional or psychological harm to an individual or group.
        'safe': Language that does not contain offensive or abusive content, and does not cause harm. """,
        "labels": ['harmful', 'safe']}
}

In [217]:
a = "non-offensive"
if a == "non-offensive":
    print("yes")
else:
    print("no")

yes


In [ ]:
# example

all_predictions = []
all_truth_labels = []
for piece in range(0, 860, 10):
    pred, truth = evaluate(prompts["prompt4"], data[piece:piece+10])
    
    print(pred)
    
    all_predictions.extend(pred)
    all_truth_labels.extend(truth) 

----- Prompt Sent -----
You are a classifier of hate speech. The following data contains language that is either harmful or safe.
        Please note these are the definitions of harmful/safe, so try to stick to these guidelines.
        'harmful': Language that is offensive, abusive, or hateful, and that can cause emotional or psychological harm to an individual or group.
        'safe': Language that does not contain offensive or abusive content, and does not cause harm. Classify each of the following sentences as 'harmful' or 'safe':

1. #ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.
2. #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendment #RNC #USMC #veterans @USER    @USER @USER @USER @USER   #fakereporting #THESWAMP #dnc #liberals @USER @USER #fakeoutrage @USER  First, it reduces the ca URL
3. #Watching #Boomer getting the news that she is still up for parole always makes me smile. #Wentworth Finale...@USER is such a trea

KeyboardInterrupt: 

In [219]:
# example

all_predictions = []
all_truth_labels = []
for piece in range(0, 300, 10):
    pred, truth = evaluate(prompts["prompt4"], data[piece:piece+10])
    
    print(pred)
    
    all_predictions.extend(pred)
    all_truth_labels.extend(truth) 

print(classification_report(all_truth_labels, all_predictions , target_names=['safe', 'harmful']))

----- Prompt Sent -----
You are a classifier of hate speech. The following data contains language that is either harmful or safe.
        Please note these are the definitions of harmful/safe, so try to stick to these guidelines.
        'harmful': Language that is offensive, abusive, or hateful, and that can cause emotional or psychological harm to an individual or group.
        'safe': Language that does not contain offensive or abusive content, and does not cause harm. Classify each of the following sentences as 'harmful' or 'safe':

1. #ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.
2. #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendment #RNC #USMC #veterans @USER    @USER @USER @USER @USER   #fakereporting #THESWAMP #dnc #liberals @USER @USER #fakeoutrage @USER  First, it reduces the ca URL
3. #Watching #Boomer getting the news that she is still up for parole always makes me smile. #Wentworth Finale...@USER is such a trea

ValueError: Found input variables with inconsistent numbers of samples: [300, 299]

In [22]:
from sklearn.metrics import classification_report
import pandas as pd

#! TODO: this is changed, doesnt run correctly, 
# need a way to loop through the prompts and use the labels with the prompt to create a table like above

def run_all_prompts(prompts, data, step=10):
    results = {}

    for pname, ptext in prompts.items():
        print(f"\n===== Running {pname} =====")

        all_predictions = []
        all_truth_labels = []

        for piece in range(0, len(data), step):
            pred, truth = evaluate(ptext, data[piece:piece+step])
            all_predictions.extend(pred)
            all_truth_labels.extend(truth)


        report = classification_report(
            all_truth_labels, 
            all_predictions, 
            target_names=[ptext['labels'][1], ptext['labels'][0]],
            output_dict=True
        )
        results[pname] = report


        print(pd.DataFrame(report).transpose())

    return results
results = run_all_prompts(prompts, data)


===== Running prompt1 =====
----- Prompt Sent -----
Classify each of the following sentences as 'offensive' or 'non-offensive':

1. #WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon ⁦@USER URL
2. #ConstitutionDay is revered by Conservatives, hated by Progressives/Socialist/Democrats that want to change it.
3. #FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendment #RNC #USMC #veterans @USER    @USER @USER @USER @USER   #fakereporting #THESWAMP #dnc #liberals @USER @USER #fakeoutrage @USER  First, it reduces the ca URL
4. #Watching #Boomer getting the news that she is still up for parole always makes me smile. #Wentworth Finale...@USER is such a treasure. URL
5. #NoPasaran: Unity demo to oppose the far-right in #London – #antifa #Oct13 — Enough is Enough! URL
6. . . . What the fuck did he do this time?
7. #RAP i

KeyboardInterrupt: 